# Compare IMUs

Kevin J. Walchko, PhD

20 June 2020

---

- bias
- bias stability
- random walk
- vibration sensitivity

## References:

- NXP Accel/Magnetometer: [FXOS8700CQ](FXOS8700CQ.pdf)
- NXP Gyro: [FXAS21002](FXAS21002.pdf)
- ST Accel/Gyro [LSM6DSOX](lsm6dsox.pdf)
- ST Accel/Gyro [LSM6DS33](lsm6ds33.pdf)
- ST Accel/Gyro [LSM9DS1 tech notes](TA0343-LSM9DS1-IMU.pdf)
- ST Accel/Gyro [LSM9DS1](lsm9ds1.pdf)
- ST Accel/Gyro [ISM330DHCX](ism330dhcx.pdf)
- InvenSense Accel/Gyro [ICM-20649](icm-20649.pdf)
- ST Magnetometer [LIS3MDL](lis3mdl.pdf)

## Terms

Power Spectral Density:

- Power: mean-squared value
- Spectral: distribution of a signal over a spectrum of frequency
- Density: mangitude of the PSD is normalized to a single Hz bandwidth

$$
RMS_{noise} = NoiseDensity \sqrt{BW * filter} \\
$$

where $bandwidth$ is half the Output Data Rate (ODR), $filter$ is a low pass filter having one of the following values: 

- 1.57(1st order)
- 1.11 (2nd order)
- 1.05(3rd order)

In [1]:
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline

import numpy as np

## Accelerometer

In [7]:
adata = {
    0 : {
        "name": "NXP_FXOS8700CQ",
        "bias": 20,
        "bits": 14,
        "noise_density": 126,
    },
    1: {
        "name": "LSM6DS33",
        "bias": 40,
        "bits": 16, 
        "noise_density": 90,  # ug/sqrt(Hz)
    },
    2: {
        "name": "LSM6DSOX",
        "bias": 20,
        "bits": 16, 
        "noise_density": 70,  # ug/sqrt(Hz)
    },
    3: {
        "name": "LSM9DS1",
        "bias": 90,
        "bits": 16, 
        "noise_density": 200,  # ug/sqrt(Hz)
    },
    4: {
        "name": "ICM-20649",
        "bias": None,
        "bits": 16, 
        "noise_density": 285,  # ug/sqrt(Hz)
    },
    5: {
        "name": "ISM330DHCX",
        "bias": None,
        "bits": 16, 
        "noise_density": 60,  # ug/sqrt(Hz)
    },
    6: {
        "name": "BNO055",
        "bias": 80,  # mg
        "bits": 14, 
        "noise_density": 150,  # ug/sqrt(Hz)
    }
}

In [13]:
accel = pd.DataFrame(adata)
accel = accel.transpose()

a_range = 2000
a_hz = 100

accel[f"rms[mg] @ {a_hz}Hz"] = accel["noise_density"]*((a_hz/2*1.57)**0.5)/1000
accel[f"mg/LSB @ {a_range//1000}G"] = 2*a_range/(2**accel["bits"])

accel.head(10)

,name,bias,bits,noise_density,rms[mg] @ 100Hz,mg/LSB @ 2G
0,NXP_FXOS8700CQ,20,14,126,1.11636,0.244141
1,LSM6DS33,40,16,90,0.797402,0.0610352
2,LSM6DSOX,20,16,70,0.620202,0.0610352
3,LSM9DS1,90,16,200,1.772,0.0610352
4,ICM-20649,None,16,285,2.52511,0.0610352
5,ISM330DHCX,None,16,60,0.531601,0.0610352
6,BNO055,80,14,150,1.329,0.244141


## Gryro

In [9]:
gdata = {
    0 : {
        "name": "NXP_FXAS21002C",
        "bits": 16,
        "noise_density": 25,  # mdps/sqrt(Hz)
        "drift": 0.02, # dps/C
    },
    1: {
        "name": "LSM6DS33",
        "bits": 16, 
        "noise_density": 7,  # mdps/sqrt(Hz)
        "drift": 0.05, # dps/C
    },
    2: {
        "name": "LSM6DSOX",
        "bits": 16, 
        "noise_density": 3.8,  # mdps/sqrt(Hz)
        "drift": 0.01, # dps/C
    },
    3: {
        "name": "ISM330DHCX",
        "bits": 16, 
        "noise_density": 5,  # mdps/sqrt(Hz)
        "drift": 0.005, # dps/C
    },
#     4: {
#         "name": "BNO055",
#         "bits": 16, 
#         "noise_density": 150,  # mdps/sqrt(Hz)
#         "drift": 0.005, # dps/C
#     }
}

In [12]:
gyro = pd.DataFrame(gdata)
gyro = gyro.transpose()

dps_range = 1000 # dps
dps_hz = 100
dps_c = 26

gyro[f"rms[mdps] @ {dps_hz}Hz"] = gyro["noise_density"]*((dps_hz/2*1.6)**0.5)
gyro[f"mdps/LSB @ {dps_range}dps"] = 2*dps_range*1000/(2**gyro["bits"])
gyro[f"dps @ {dps_c}C"] = dps_c*gyro["drift"]

gyro.head(10)

,name,bits,noise_density,drift,rms[mdps] @ 100Hz,mdps/LSB @ 1000dps,dps @ 26C
0,NXP_FXAS21002C,16,25,0.02,223.607,30.5176,0.52
1,LSM6DS33,16,7,0.05,62.6099,30.5176,1.3
2,LSM6DSOX,16,3.8,0.01,33.9882,30.5176,0.26
3,ISM330DHCX,16,5,0.005,44.7214,30.5176,0.13


## Magnetometer